In [1]:
## IMPORTS
import random
from deap import base
from deap import creator
from deap import tools
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


In [3]:
def generate_random_num_attr():
  original = np.array([0.3552483393772611, 0.7578300933136081, 0.09057686913234943, 1.033179934185795, 1.2749134898394006, 0.9592455936953672, 0.9775812152327962, 0.9583027148594099])
  noises = []
  sum_list = []

  noises.append(np.random.normal(0, 0.5))
  noises.append(np.random.normal(0, 1))
  noises.append(np.random.normal(0, 0.1))
  noises.append(np.random.normal(0, 0.01))
  noises.append(np.random.normal(0, 0.1))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.1))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [4]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
def evalOneMax(individual):
    print(individual[0])
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-09", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in yf.tickers_sp500():
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    print(transactionCopy.iloc[-1]['total'])
    return transactionCopy.iloc[-1]['total'],

In [6]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [7]:
def main():


    pop = toolbox.population(n=512)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    # Begin the evolution
    for g in range(9):
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [8]:
main()

[0.5011440230323366, 3.8034271746341566, 0.17302929388129204, 1.0355600122140736, 1.2191151180654558, 0.9385543909714653, 0.9752430891941133, 0.8777836122222266]
1542186.6032520207
[0.8254899284718041, 0.5119391395804193, 0.18845027094399577, 1.0188675226373216, 1.0160165032640347, 1.0097721954414813, 0.960185220326852, 0.920481787141893]
21790.303449467116
[2.0161950738176095, 0.03479839795641593, 0.22523892491314468, 1.03135879694148, 1.1204057428449965, 1.0112519374222637, 1.0348245300406709, 0.9047673560295063]
199074.65138944593
[1.0971769517749117, 0.5631783401333692, 0.10986620852780997, 1.028847033850841, 1.0867402886409883, 1.031863708523853, 1.0788892557080707, 0.8420444905173192]
103158.51174715594
[0.3208438301370652, 0.18314785771270503, 0.2209531282521968, 1.063462268237701, 1.0059115125652938, 1.0310739166398373, 1.0383808147305944, 0.9595601650352118]
1572.717150767987
[1.7645814112018412, 0.8406418667247287, -0.15094342973192656, 1.0291216642535883, 1.1391805372937256,

KeyError: 'volumePerc'